In [2]:
-- 28-1. Restoring a Database from a Full Backup

-- The first example in this recipe is a simple RESTORE from the latest backup set on the device (in this example, two backup sets exist on the device for the TestDB database, and you want the second one). For the demonstration, I’ll start by creating two full backups on a single device.

USE master
;
GO
Declare @BackupDate Char(8) = Convert(Varchar,GetDate(),112)
,@BackupPath Varchar(50);
Set @BackupPath = 'C:\Apress\TestDB_' + @BackupDate + '.BAK';
BACKUP DATABASE TestDB
TO DISK = @BackupPath;
GO
-- Time passes, we make another backup to the same device
USE master;
GO
Declare @BackupDate Char(8) = Convert(Varchar,GetDate(),112)
,@BackupPath Varchar(50);
Set @BackupPath = 'C:\Apress\TestDB_' + @BackupDate + '.BAK';
BACKUP DATABASE TestDB
TO DISK = @BackupPath;
GO

Commands completed successfully.

Processed 368 pages for database 'TestDB', file 'TestDB' on file 1.

Processed 1 pages for database 'TestDB', file 'TestDB_log' on file 1.

BACKUP DATABASE successfully processed 369 pages in 0.143 seconds (20.115 MB/sec).

Commands completed successfully.

Processed 368 pages for database 'TestDB', file 'TestDB' on file 2.

Processed 1 pages for database 'TestDB', file 'TestDB_log' on file 2.

BACKUP DATABASE successfully processed 369 pages in 0.062 seconds (46.394 MB/sec).

Total execution time: 00:00:01.531

In [3]:
-- Now the database is restored using the second backup from the device (notice that the REPLACE argument is used to tell SQL Server to overlay the existing TestDB database).

USE master;
GO
Declare @DeviceName Varchar(50);
Select @DeviceName = b.physical_device_name
From msdb.dbo.backupset a
INNER JOIN msdb.dbo.backupmediafamily b
ON a.media_set_id = b.media_set_id
Where a.database_name = 'TestDB'
And a.type = 'D'
And Convert(Varchar,a.backup_start_date,112) = Convert(Varchar,GetDate(),112);
RESTORE DATABASE TestDB
FROM DISK = @DeviceName
WITH FILE = 2, REPLACE;
GO

Commands completed successfully.

Processed 368 pages for database 'TestDB', file 'TestDB' on file 2.

Processed 1 pages for database 'TestDB', file 'TestDB_log' on file 2.

RESTORE DATABASE successfully processed 369 pages in 0.068 seconds (42.300 MB/sec).

Total execution time: 00:00:01.556

In [4]:
-- In this second example, a new database is created by restoring from the TestDB backup, creating a new database called TrainingDB. Notice that the MOVE argument is used to designate the location of the new database files.

USE master;
GO
Declare @DeviceName Varchar(50);
Select @DeviceName = b.physical_device_name
From msdb.dbo.backupset a
INNER JOIN msdb.dbo.backupmediafamily b
ON a.media_set_id = b.media_set_id
Where a.database_name = 'TestDB'
And a.type = 'D'
And Convert(Varchar,a.backup_start_date,112) = Convert(Varchar,GetDate(),112);
RESTORE DATABASE TrainingDB
FROM DISK = @DeviceName
WITH FILE = 2,
MOVE 'TestDB' TO 'C:\Apress\TrainingDB.mdf',
MOVE 'TestDB_log' TO 'C:\Apress\TrainingDB_log.LDF';
GO

Commands completed successfully.

Processed 368 pages for database 'TrainingDB', file 'TestDB' on file 2.

Processed 1 pages for database 'TrainingDB', file 'TestDB_log' on file 2.

RESTORE DATABASE successfully processed 369 pages in 0.042 seconds (68.487 MB/sec).

Total execution time: 00:00:00.469

In [5]:
-- In the last example for this recipe, the TestDB database is restored from a striped backup set. First, I create a backup set that will be used to perform the restore of a striped backup set.

USE master;
GO
/* The path for each file should be changed to a path matching one
That exists on your system. */
BACKUP DATABASE TestDB
TO DISK = 'C:\Apress\Recipes\TestDB_Stripe1.bak'
, DISK = 'D:\Apress\Recipes\TestDB_Stripe2.bak'
, DISK = 'E:\Apress\Recipes\TestDB_Stripe3.bak'
WITH NOFORMAT, NOINIT,
NAME = N'TestDB-Stripe Database Backup',
SKIP, STATS = 20;
GO

Commands completed successfully.

: Msg 3201, Level 16, State 1, Line 7
Cannot open backup device 'C:\Apress\Recipes\TestDB_Stripe1.bak'. Operating system error 3(The system cannot find the path specified.).

: Msg 3013, Level 16, State 1, Line 7
BACKUP DATABASE is terminating abnormally.

Total execution time: 00:00:00.031

In [6]:
-- Now, I will perform the restore of the striped backup set.

USE master;
GO
/* You should use the same file path for each file as specified
in the backup statement. */
RESTORE DATABASE TestDB
FROM DISK = 'C:\Apress\Recipes\TestDB_Stripe1.bak'
, DISK = 'D:\Apress\Recipes\TestDB_Stripe2.bak'
, DISK = 'E:\Apress\Recipes\TestDB_Stripe3.bak'
WITH FILE = 1, REPLACE;
GO

Commands completed successfully.

: Msg 3201, Level 16, State 2, Line 7
Cannot open backup device 'C:\Apress\Recipes\TestDB_Stripe1.bak'. Operating system error 3(The system cannot find the path specified.).

: Msg 3013, Level 16, State 1, Line 7
RESTORE DATABASE is terminating abnormally.

Total execution time: 00:00:00.030

In [7]:
-- 28-2. Restoring a Database from a Transaction Log Backup

-- For this demonstration, the TrainingDB created in the previous recipe will be used (if it doesn’t exist, we will create it).

USE master;
GO
IF NOT EXISTS (SELECT name FROM sys.databases
WHERE name = 'TrainingDB')
BEGIN
CREATE DATABASE TrainingDB;
END
GO
-- Add a table and some data to it
USE TrainingDB
GO
SELECT *
INTO dbo.SalesOrderDetail
FROM AdventureWorks2014.Sales.SalesOrderDetail;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(121317 rows affected)

Total execution time: 00:00:00.429

In [8]:
-- This database will be given a full backup and two consecutive transaction log backups.

USE master;
GO
Declare @BackupDate Char(8) = Convert(Varchar,GetDate(),112)
,@BackupPath Varchar(50);
Set @BackupPath = 'C:\Apress\TrainingDB_' + @BackupDate + '.BAK';
BACKUP DATABASE TrainingDB
TO DISK = @BackupPath;
GO
BACKUP LOG TrainingDB
TO DISK = 'C:\Apress\TrainingDB_20120430_8AM.trn';
GO
-- Two hours pass, another transaction log backup is made
BACKUP LOG TrainingDB
TO DISK = 'C:\Apress\TrainingDB_20120430_10AM.trn';
GO

Commands completed successfully.

Processed 1888 pages for database 'TrainingDB', file 'TestDB' on file 1.

Processed 1 pages for database 'TrainingDB', file 'TestDB_log' on file 1.

BACKUP DATABASE successfully processed 1889 pages in 0.108 seconds (136.587 MB/sec).

Processed 1562 pages for database 'TrainingDB', file 'TestDB_log' on file 1.

BACKUP LOG successfully processed 1562 pages in 0.091 seconds (134.030 MB/sec).

Processed 2 pages for database 'TrainingDB', file 'TestDB_log' on file 1.

BACKUP LOG successfully processed 2 pages in 0.016 seconds (0.549 MB/sec).

Total execution time: 00:00:00.363

In [9]:
-- The previous RESTORE examples have assumed that there were no existing connections in the database to be restored over. However, in this example, I demonstrate how to kick out any connections to the database prior to performing the RESTORE.

USE master;
GO
-- Kicking out all other connections
ALTER DATABASE TrainingDB
SET SINGLE_USER
WITH ROLLBACK IMMEDIATE;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.098

In [13]:
-- Next, a database backup and two transaction log backups are restored from backup.

USE master;
GO
Declare @DeviceName Varchar(50);
Select @DeviceName = b.physical_device_name
From msdb.dbo.backupset a
INNER JOIN msdb.dbo.backupmediafamily b
ON a.media_set_id = b.media_set_id
Where a.database_name = 'TrainingDB'
And a.type = 'D'
And Convert(Varchar,a.backup_start_date,112) = Convert(Varchar,GetDate(),112);
RESTORE DATABASE TrainingDB
FROM DISK = @DeviceName
WITH NORECOVERY, REPLACE;
RESTORE LOG TrainingDB
FROM DISK = 'C:\Apress\ TrainingDB_20120430_8AM.trn'
WITH NORECOVERY, REPLACE
RESTORE LOG TrainingDB
FROM DISK = 'C:\Apress\ TrainingDB_20120430_10AM.trn'
WITH RECOVERY, REPLACE

Commands completed successfully.

Processed 1888 pages for database 'TrainingDB', file 'TestDB' on file 1.

Processed 1 pages for database 'TrainingDB', file 'TestDB_log' on file 1.

RESTORE DATABASE successfully processed 1889 pages in 0.222 seconds (66.448 MB/sec).

: Msg 3201, Level 16, State 2, Line 16
Cannot open backup device 'C:\Apress\ TrainingDB_20120430_8AM.trn'. Operating system error 2(The system cannot find the file specified.).

: Msg 3013, Level 16, State 1, Line 16
RESTORE LOG is terminating abnormally.

: Msg 3201, Level 16, State 2, Line 19
Cannot open backup device 'C:\Apress\ TrainingDB_20120430_10AM.trn'. Operating system error 2(The system cannot find the file specified.).

: Msg 3013, Level 16, State 1, Line 19
RESTORE LOG is terminating abnormally.

Total execution time: 00:00:00.466

In [11]:
-- In this second example, I'll use STOPAT to restore the database and transaction log as of a specific point in time. To demonstrate, first a full backup will be taken of the TrainingDB database.

USE master;
GO
BACKUP DATABASE TrainingDB
TO DISK = 'C:\Apress\TrainingDB_StopAt.bak';
GO

Commands completed successfully.

: Msg 927, Level 14, State 2, Line 5
Database 'TrainingDB' cannot be opened. It is in the middle of a restore.

: Msg 3013, Level 16, State 1, Line 5
BACKUP DATABASE is terminating abnormally.

Total execution time: 00:00:00.013

In [12]:
-- Next, rows will be deleted out of the table, and the current time after the change will be queried.

USE TrainingDB;
GO
DELETE dbo.SalesOrderDetail
WHERE ProductID = 776;
GO
SELECT GETDATE();
GO

: Msg 927, Level 14, State 2, Line 3
Database 'TrainingDB' cannot be opened. It is in the middle of a restore.

: Msg 208, Level 16, State 1, Line 5
Invalid object name 'dbo.SalesOrderDetail'.

(1 row affected)

Total execution time: 00:00:00.612

(No column name)
2021-01-14 17:47:10.977


In [ ]:
-- Next, a transaction log backup is performed.

BACKUP LOG TrainingDB
TO DISK = 'C:\Apress\TrainingDB_20120430_2022.trn';
GO

In [ ]:
-- The database is restored from backup, leaving it in NORECOVERY so that the transaction log backup can also be restored.

USE master;
GO
RESTORE DATABASE TrainingDB
FROM DISK = 'C:\Apress\TrainingDB_StopAt.bak'
WITH FILE = 1, NORECOVERY,
STOPAT = '2012-04-30 22:17:10.563';
GO

In [ ]:
-- Next, the transaction log is restored, also designating the time prior to the data deletion.

RESTORE LOG TrainingDB
FROM DISK = 'C:\Apress\TrainingDB_20120430_2022.trn'
WITH RECOVERY,
STOPAT = '2012-04-30 22:17:10.563';
GO

In [ ]:
-- The following query confirms that you have restored just prior to the data deletion:

USE TrainingDB;
GO
SELECT COUNT(*)
FROM dbo.SalesOrderDetail
WHERE ProductID = 776;
GO

In [ ]:
-- 28-3. Restoring a Database from a Differential Backup

-- First, however, I’ll set up the example by performing a full, differential, and transaction log backup on the TrainingDB database.

USE master;
GO
BACKUP DATABASE TrainingDB
TO DISK = 'C:\Apress\TrainingDB_DiffExample.bak';
GO
-- Time passes
BACKUP DATABASE TrainingDB
TO DISK = 'C:\Apress\TrainingDB_DiffExample.diff'
WITH DIFFERENTIAL;
GO

-- More time passes
BACKUP LOG TrainingDB
TO DISK = 'C:\Apress\TrainingDB_DiffExample_tlog.trn';
GO

In [ ]:
-- Now, I’ll demonstrate performing a RESTORE, bringing the database back to the completion of the last transaction log backup.

USE master;
GO
-- Full database restore
RESTORE DATABASE TrainingDB
FROM DISK = 'C:\Apress\TrainingDB_DiffExample.bak'
WITH NORECOVERY, REPLACE;
GO
-- Differential
RESTORE DATABASE TrainingDB
FROM DISK = 'C:\Apress\TrainingDB_DiffExample.diff'
WITH NORECOVERY;
GO
-- Transaction log
RESTORE LOG TrainingDB
FROM DISK = 'C:\Apress\TrainingDB_DiffExample_tlog.trn'
WITH RECOVERY;
GO

In [ ]:
-- 28-4. Restoring a File or Filegroup

USE master;
GO
If Not Exists (Select name from sys.databases where name = 'VLTestDB')
Begin
CREATE DATABASE VLTestDB
ON PRIMARY
    ( NAME = N'VLTestDB',FILENAME = N'c:\Apress\VLTestDB.mdf'
    ,SIZE = 4072 KB , FILEGROWTH = 0 ),
FILEGROUP FG2
    ( NAME = N'VLTestDB2', FILENAME = N'c:\Apress\VLTestDB2.ndf'
    , SIZE = 3048 KB , FILEGROWTH = 1024 KB )
    ,( NAME = N'VLTestDB3', FILENAME = N'c:\Apress\VLTestDB3.ndf'
    , SIZE = 3048 KB , FILEGROWTH = 1024 KB )
LOG ON
    ( NAME = N'VLTestDBLog', FILENAME = N'c:\Apress\VLTestDB_log.ldf'
    , SIZE = 1024 KB , FILEGROWTH = 10 %);
Alter DATABASE VLTestDB
Modify FILEGROUP FG2 Default;
END
GO
USE master;
GO
BACKUP DATABASE VLTestDB
FILEGROUP = 'FG2'
TO DISK = 'C:\Apress\VLTestDB_FG2.bak'
WITH NAME = N'VLTestDB-Full Filegroup Backup',
SKIP, STATS = 20;
GO

In [ ]:
-- Time passes, and then a transaction log backup is taken for the database.

BACKUP LOG VLTestDB
TO DISK = 'C:\Apress\VLTestDB_FG_Example.trn';
GO

In [ ]:
-- Next, the database filegroup FG2 is restored from backup, followed by the restore of a transaction log backup.

USE master;
GO
RESTORE DATABASE VLTestDB
FILEGROUP = 'FG2'
FROM DISK = 'C:\Apress\VLTestDB_FG2.bak'
WITH FILE = 1, NORECOVERY, REPLACE;
RESTORE LOG VLTestDB
FROM DISK = 'C:\Apress\VLTestDB_FG_Example.trn'
WITH FILE = 1, RECOVERY;
GO

In [ ]:
-- 28-5. Performing a Piecemeal (PARTIAL) Restore

-- First, to set up this example, the primary and FG2 filegroups in the VLTestDB are backed up.

USE master;
GO
BACKUP DATABASE VLTestDB
FILEGROUP = 'PRIMARY'
TO DISK = 'C:\Apress\VLTestDB_Primary_PieceExmp.bak';
GO
BACKUP DATABASE VLTestDB
FILEGROUP = 'FG2'
TO DISK = 'C:\Apress\VLTestDB_FG2_PieceExmp.bak';
GO

In [ ]:
-- After that, a transaction log backup is performed.

BACKUP LOG VLTestDB
TO DISK = 'C:\Apress\VLTestDB_PieceExmp.trn';
GO

In [ ]:
-- Next, a piecemeal RESTORE is performed, recovering just the PRIMARY filegroup.

USE master;
GO
RESTORE DATABASE VLTestDB
FILEGROUP = 'PRIMARY'
FROM DISK = 'C:\Apress\VLTestDB_Primary_PieceExmp.bak'
WITH PARTIAL, NORECOVERY, REPLACE;
RESTORE LOG VLTestDB
FROM DISK = 'C:\Apress\VLTestDB_PieceExmp.trn'
WITH RECOVERY;
GO

In [ ]:
-- The other filegroup, FG2, now contains unavailable files. You can view the file status by querying sys.database_files from the VLTestDB database.

USE VLTestDB;
GO
SELECT name,state_desc
FROM sys.database_files;
GO

In [ ]:
-- 28-6. Restoring a Page

-- To set up this example, a full database backup is created for the TestDB database.

USE master;
GO
BACKUP DATABASE TestDB
TO DISK = 'C:\Apress\TestDB_PageExample.bak';
GO

In [ ]:
-- Next, a restore is performed using the PAGE argument.

USE master;
GO
RESTORE DATABASE TestDB
PAGE = '1:8'
FROM DISK = 'C:\Apress\TestDB_PageExample.bak'
WITH NORECOVERY, REPLACE;
GO

In [ ]:
-- At this point, any differential or transaction log backups taken after the last full backup should also be restored. Since there were none in this example, no further backups are restored.
-- Next, and this is something that departs from previous examples, a new transaction log backup must be created that captures the restored page.

BACKUP LOG TestDB
TO DISK = 'C:\Apress\TestDB_PageExample_tlog.trn';
GO

In [ ]:
-- To finish the page restore process, the latest transaction log taken after the RESTORE. . .PAGE must be executed with RECOVERY.

RESTORE LOG TestDB
FROM DISK = 'C:\Apress\TestDB_PageExample_tlog.trn'
WITH RECOVERY;

In [ ]:
-- 28-7. Identifying Databases with Multiple Recovery Paths

-- This recipe demonstrates how to use the sys.database_recovery_status catalog view to get information about a database with more than one recovery path. In the first step, I will create a new database and give it a full database backup, create a table and some rows, and finish up with a transaction log backup.

USE master;
GO
IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'RomanHistory')
BEGIN
CREATE DATABASE RomanHistory;
END
GO
BACKUP DATABASE RomanHistory
TO DISK = 'C:\Apress\RomanHistory_A.bak';
GO

In [ ]:
USE RomanHistory;
GO
CREATE TABLE EmperorTitle
(EmperorTitleID int NOT NULL PRIMARY KEY IDENTITY(1,1), TitleNM varchar(255));
GO
INSERT Into EmperorTitle (TitleNM)
VALUES ('Aulus'), ('Imperator'), ('Pius Felix'), ('Quintus');
BACKUP LOG RomanHistory
TO DISK = 'C:\Apress\RomanHistory_A.trn';
GO

In [ ]:
-- Next, I’ll query the sys.database_recovery_status catalog view to get information about the database at this point (column aliases are used to shorten the names for presentation in this book).

USE msdb;
GO
SELECT LastLSN = last_log_backup_lsn ,Rec_Fork = recovery_fork_guid
,Frst_Fork = first_recovery_fork_guid ,Fork_LSN = fork_point_lsn
FROM sys.database_recovery_status
WHERE database_id = DB_ID('RomanHistory');
GO

In [ ]:
-- Next, I will perform a few more data modifications and another transaction log backup.

USE RomanHistory;
GO
INSERT Into EmperorTitle (TitleNM)
VALUES ('Germanicus'), ('Lucius'), ('Maximus'), ('Titus');
GO
BACKUP LOG RomanHistory
TO DISK = 'C:\Apress\RomanHistory_B.trn';
GO

In [ ]:
-- I’ll now go ahead and RESTORE the database to a prior state (but not to the latest state).

USE master;
GO
RESTORE DATABASE RomanHistory
FROM DISK = 'C:\Apress\RomanHistory_A.bak'
WITH NORECOVERY, REPLACE;
RESTORE DATABASE RomanHistory
FROM DISK = 'C:\Apress\RomanHistory_A.trn'
WITH RECOVERY, REPLACE;
GO

In [ ]:
-- Now if I reissue the previous query against sys.database_recovery_status, I will see that both the fork_point_lsn and first_recovery_fork_guid columns are no longer NULL.

USE msdb;
GO
SELECT LastLSN = last_log_backup_lsn ,Rec_Fork = recovery_fork_guid
,Frst_Fork = first_recovery_fork_guid ,Fork_LSN = fork_point_lsn
FROM sys.database_recovery_status
WHERE database_id = DB_ID('RomanHistory');
GO